<a href="https://colab.research.google.com/github/KorStats/ADP-test/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이상치 확인
def outliers_iqr(dt, col):
  quantile_1, quantile_3=np.percentile(dt[col], [25, 75])
  iqr=quantile_3-quantile_1
  lower_whis=quantile_1-(iqr*1.5)
  upper_whis=quantile_3+(iqr*1.5)
  outliers=dt[(dt[col] > upper_whis) | (dt[col] < lower_whis)]
  return outliers[[col]]

outliers=outliers_iqr(df, 'color_intensity')
drop_outliers=df.drop(index=ouliers.index) #이상치 제거
df['color_intensity'].fillna(df['color_intensity'].mean()) #이상치 대체
df_dummy=pd.get_dummies(df, columns=['Class']) #범주형 변수 처리

In [ ]:
from sklearn.medel_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.1, random_state=0, stratify='y')

In [ ]:
#StandardScaler : 평균이0, 분산이 1인 정규분포로 스케일링, 이상치에 매우 민감, 분류분석에 유용
#MinMaxScaler : 0과 1사이의 값으로 스케일링, 이상치에 매우 민감, 회귀분석에 유용
#MaxAbsScaler : -1과 1사이의 값으로 스케일링, 이상치에 매우 민감, 회귀분석에 유용
#RobustScaler : 중앙값과 사분위값을 활용, 이상치 영향 최소화

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
Scaler=StandardScaler() / MinMaxScaler() / MaxAbsScaler() / RobustScaler()
X_train_sc=scaler.fit_transform(X_train)
X_test_sc=scaler.transform(X_test)
X_Original=scaler.inverse_transform(X_train_sc) #원본으로 변경

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

x=StandardScaler().fit_transform(x)
pca=PCA(n_components=4)
pca_fit=pca.fit(x)
pca.singular_values_ #고유값
pca.explained_variance_ratio_ #분산설명력
plt.plot(pca.explained_variance_ratio_, 'o-') #Scree Plot
plt.title('Scree Plot') / plt.xlabel('Number of Components') / plt.ylabel('Cumulative Explained Variance')

pca=PCA(n_components=2) #주성분 갯수 2개 생성
principalComponents=pca.fit_transform(x)
principal_df=pd.DataFrame(data=principalComponents, columns=['pc1', 'pd2'])
principal_df.head()
sns.scatterplot(x='pc1', y='pc2', hue=df.Class, data=principal_df) #주성분 산포도
plt.show()

In [ ]:
!pip install imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from matplotlib import pyplot as plt
import seaborn as sns

undersample=RandomUnderSampler(sampling_strategy='majority')
oversample=RandomOverSampler(sampling_strategy=0.5)
smote_sample=SMOTE(sampling_strategy='minority')
x_under, y_under=undersample.fit_resample(x, y)
x_over, y_over=oversample.fit_resample(x, y)
x_sm, y_sm=smote_sample.fit_resample(x, y)

fig, axes=plt.subplots(nrows=2, ncols=2, figsize=(10,10))
sns.scatterplot(x[:,1], x[:,2], hue=y, ax=axes[0][0], alpha=0.5)
sns.scatterplot(x_under)[:,1], x_under[:,2], hue=y_under, ax=axes[0][1], alpha=0.5)
sns.scatterplot(x_over[:,1], x_over[:,2], hue=y_over, ax=axes[1][0], alpha=0.5)
sns.scatterplot(x_sm[:,1], x_sm[:,2], hue=y_sm, ax=axes[1][1], alpha=0.5)
axes[0][0].set_title('Original Data')
axes[0][1].set_title('Random Under Sampling')
axes[1][1].set_title('SMOTE')
plt.show()